# Spectro pulsé RO
## freq_LO = §FREQ§GHz

In [ ]:
f.close()
f = h5py.File(§HDF5_PATH§, 'r')
#print(f.keys())

fr = f['parameters']['freq'][()]
data = (f['data']['I'][()] + 1j*f['data']['Q'][()]) * np.exp(1j*fr*94*2*np.pi)
f.close()
#print(data.shape)

phi = np.unwrap(np.angle(data))
phi = phi-(np.amin(phi)+np.amax(phi))/2

In [ ]:
def atan(fr, f0, a, b):
    return a*np.arctan(b*(fr-f0))

popt, pcov = opt.curve_fit(atan, fr, phi, (5, -2, 1))
print('f0, a, b =', popt)

fig, ax = plt.subplots()
ax.plot(fr, phi, '.-')
ax.plot(fr, atan(fr, *popt))
ax.legend(('Data', f'Fit: f = {popt[0]:.6f} GHz'));

In [ ]:
fig, ax = plt.subplots()
ax.plot(data.real, data.imag, '.-')
plt.axis('equal');

In [ ]:
_opt = popt
_cov = pcov
_result = {'freq': popt[0]}
_result